In [ ]:
import os
import numpy as np
import pandas as pd
import pywt
import cv2
from pathlib import Path

def compute_dwt_features(face_image_path):
    img = cv2.imread(str(face_image_path))
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    coeffs2 = pywt.dwt2(img_gray, 'haar')
    LL, (LH, HL, HH) = coeffs2  

    LL_energy = np.sum(np.square(LL))
    LH_energy = np.sum(np.square(LH))
    HL_energy = np.sum(np.square(HL))
    HH_energy = np.sum(np.square(HH))

    print(f"Processing {face_image_path.name}, LL: {LL_energy}, LH: {LH_energy}, HL: {HL_energy}, HH: {HH_energy}")

    return LL_energy, LH_energy, HL_energy, HH_energy

def extract_video_features(video_folder_path, folder_name):
    features_data = []
    image_files = sorted(Path(video_folder_path).glob("*.png"))

    if not image_files:
        print(f"No images found in {video_folder_path}. Skipping...")
        return None

    for image_file in image_files:
        LL_energy, LH_energy, HL_energy, HH_energy = compute_dwt_features(image_file)
        features_data.append([folder_name, image_file.name, LL_energy, LH_energy, HL_energy, HH_energy])

    return features_data

base_dir = Path(r"...")
video_folders = ['original', 'Deepfakes', 'Face2Face', 'FaceShifter', 'FaceSwap', 'NeuralTextures']

for folder in video_folders:
    folder_path = base_dir / folder
    features_data = []

    result = extract_video_features(folder_path, folder)
    if result:
        features_data.extend(result)

    if features_data:
        df = pd.DataFrame(features_data, columns=['Folder', 'Image', 'LL Energy', 'LH Energy', 'HL Energy', 'HH Energy'])
        csv_file_path = base_dir / f'{folder}_dwt_features.csv'
        df.to_csv(csv_file_path, index=False)
        print(f"Features saved for {folder} at {csv_file_path}")
    else:
        print(f"No valid images processed for {folder}. No data saved.")
